In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as colors
import numpy as np
from scipy import interpolate, stats

In [2]:
import ProcessOT2DOE as process

c:\users\edwin\appdata\local\programs\python\python36\lib\site-packages\oauth2client\_helpers.py:255: UserWarning: Cannot access OT2creds.txt: No such file or directory
  warnings.warn(_MISSING_FILE_MESSAGE.format(filename))


# Step 1: Load Sample Information
* Will be in the form of a dataframe. This dataframe should be standardized in the original documentation.

In [ ]:
sample_info = pd.read_csv(r"C:\Users\Edwin\Desktop\02_26_21_OuzoScan\Sample_info") # make it find the blank position from sample_info
sample_info

# Step 2: Load UV-Vis Date
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. **NOTE**: Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
* ***If loading from single cuvette reader***: 

* issue when over scanned on accident 

In [ ]:
def merge_wavelength_dfs(df_list):
    merge_list = []
    for i, df in enumerate(df_list):
        if i == 0:
            df = df
        else: 
            df = df.drop(['Wavelength'])
        print(len(df))
        merge_list.append(df)
    return pd.concat(merge_list)

In [ ]:
path = r"C:\Users\Edwin\Desktop\02_26_21_OuzoScan\02_26_21_OuzoScan\02_26_21_OuzoScan.xlsx"
plate_names = ['Sheet1','Sheet2','Sheet3','Sheet4','Sheet5']
plate_dfs = process.extract_plates(path, plate_names) # can edit/remove wells accidently measured etc, but really should be done at excel level
merged_df = merge_wavelength_dfs(plate_dfs) # a check
merged_df[148:]

# Step 2: Select/Locate Blank 
* ***If loading from plate reader***: Once the plate has been read, allow for exporting to an excel sheet and reformat for easy upload and merging.
    * The plate reader should automatically ask you if you would like to export to excel. When running multiple plates you can just keep selecting export and it will automatically add the new run as an additional sheet to the originally generated excel sheet. 
    * To format for easy loading:
        1. Create a new sheet in the same exported excel file and name it something simple (i.e. sheet1). 
        2. Copy the data from the sheet and add it to the newly created sheet. **NOTE**: Make sure to paste in the data as "only values". 
        3. Repeat for every single exported sheet (i.e. every plate)
* ***If loading from single cuvette reader***: 

In [ ]:
merged_df

In [ ]:
# baseline and combine
# baseline_series = merged_df.iloc[-1]
# merged_baselined_df = baseline_correction(merged_df, baseline_series)
combined_df = process.add_abs_to_sample_info(sample_info, merged_df)
combined_df

In [ ]:
combined_df.to_csv(r"C:\Users\Edwin\Desktop\OT2-DOE\02_26_21_OuzoScan\combined_data", index = False, header=True)

In [ ]:
test_open = pd.read_csv(r"C:\Users\Edwin\Desktop\11_18_2020\11_18_20_merged_info")
test_open

In [ ]:
# extract data by dict method of df calling series
wavelength = '400.0nm'
x_name = combined_df['Component 4 wtf'][0]
y_name = combined_df['Component 3 wtf'][0]
x = [float(i) for i in combined_df['Component 4 wtf'][1:].tolist()][:-1] #ethanol, 
y = [float(i) for i in combined_df['Component 3 wtf'][1:].tolist()][:-1] # pfh
z = [float(i) for i in combined_df[wavelength][1:].tolist()][:-1]
combined_restricted_xyz = [x,y,z]
modi = remove_visual_outliers(x,y,z,2) # this should only be used to find the new vmin and vmax but not to exclude